<h1 style="color:blue"> Chapter 10 </h1>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
%matplotlib inline

In [ ]:
stocks=["RELIANCE.NS","SBIN.NS","TITAN.NS"]

In [ ]:
prices=pd.DataFrame()

In [ ]:
for stock in stocks:
    prices[stock]=yf.download(stock,start="2010-01-01",end="2020-01-02")['Close']

In [ ]:
prices.head()

In [ ]:
stock_returns = prices.pct_change().dropna()
stock_returns.head()

In [ ]:
weights = [0.2,0.3,0.5]

In [ ]:
def port_returns(r = stock_returns,w = weights):
    r1 = np.cumprod(r.add(1)).mul(w).sum(axis=1).sub(1)
    return r1

In [ ]:
wt = port_returns()
wt.head()

In [ ]:
stock_returns.mul(weights).sum(axis=1)[:5]

In [ ]:
wt.add(1).cumprod().plot();

In [ ]:
stock_returns.cov()

In [ ]:
import trafalgar as tf
import math # needed for the trafalgar library

In [ ]:
tf.graph_close(["FB", "AAPL", "TSLA"], "2020-01-01", "2022-01-01")

In [ ]:
tf.graph_returns(["ITC.NS", "RELIANCE.NS"], (0.5,0.5),"2020-01-01", "2022-01-01")

In [ ]:
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting
from pypfopt import CLA
from pypfopt import hierarchical_portfolio

In [ ]:
nifty = pd.read_csv("Nifty.csv")

In [ ]:
nifty.Date = pd.to_datetime(nifty.Date,format="%d-%b-%y")
nifty.set_index('Date',inplace=True)
nifty.head(2)

In [ ]:
ret = expected_returns.mean_historical_return(nifty)
risk = risk_models.sample_cov(nifty)

In [ ]:
nifty_opt = EfficientFrontier(ret, risk)
raw_weights = nifty_opt.max_sharpe()
nifty_opt.portfolio_performance(verbose=True)

In [ ]:
cla = CLA(ret,risk)

In [ ]:
plotting.plot_efficient_frontier(cla)

In [ ]:
nifty_opt_short = EfficientFrontier(ret, risk,weight_bounds=(-1,1))
raw_weights = nifty_opt_short.max_sharpe()
nifty_opt_short.portfolio_performance(verbose=True)

In [ ]:
port = cla.efficient_frontier(200)

In [ ]:
sns.lineplot(x = port[1],y=port[0])
plt.title("Efficient Frontier")
plt.xlabel("Portfolio Standard Deviation")
plt.ylabel("POrtfolio Return");

In [ ]:
from pypfopt import BlackLittermanModel

In [ ]:
risk = risk_models.sample_cov(nifty)
my_views = {'Asian Paints Ltd.': 0.20, 'I T C Ltd.': -0.30, 'Shree Cement Ltd.': 0.3, 
            'Reliance Industries Ltd.': 0.4}
bl = BlackLittermanModel(risk, pi="equal", absolute_views=my_views)
rets = bl.bl_returns()

ef = EfficientFrontier(rets, risk)
ef.max_sharpe()

In [ ]:
nifty_hc = hierarchical_portfolio.HRPOpt(nifty)

In [ ]:
nifty_hc.plot_dendrogram();

In [ ]:
import pandas as pd
from mftool import Mftool 

In [ ]:
mf = Mftool()

In [ ]:
schemes = mf.get_scheme_codes()
len(schemes)

In [ ]:
schemes['112938']

In [ ]:
nippon = mf.get_scheme_historical_nav("112938",as_Dataframe=True)
nippon.head(3)

In [ ]:
def cppi(asset,rf=0,floor=0.9,cushion_multiple=5):
  n = asset.shape[0]
  port_value = []
  port_value.append(asset[0])
  cushion = port_value[0] * (1-floor)
  risky_asset = cushion * cushion_multiple
  risk_free_asset = port_value[0] - risky_asset
  
  for i in range(1,n):
    port_value.append((asset[i]/asset[i-1])*risky_asset + risk_free_asset*(1+rf))
    cushion_new = port_value[i] - port_value[0]*floor
    risky_asset = cushion_new * cushion_multiple
    risk_free_asset = port_value[i] - risky_asset 
   
  return port_value


In [ ]:
sensex = yf.download("^BSESN")
sensex.head(2)

In [ ]:
sensex_ad = sensex['Adj Close']

In [ ]:
sensex['CPPI'] = cppi(sensex_ad)

In [ ]:
sensex.iloc[0:100,[4,6]].head()

In [ ]:
sensex.iloc[0:100,[4,6]].plot()
plt.title("Sensex vs. CPPI: 1997");

In [ ]:
def find_nd1(S,K,rf,t,vol):
  d1 = (np.log(S/K)+(rf+vol*vol*0.5)*t)/(vol*np.sqrt(t))
  nd1 = scipy.stats.norm.cdf(d1)
  return nd1


def find_nd2(S,K,rf,t,vol):
  d1 = (np.log(S/K)+(rf+vol*vol*0.5)*t)/(vol*np.sqrt(t))
  d2 = d1 - vol*np.sqrt(t)
  nd2 = scipy.stats.norm.cdf(d2)
  return nd2

In [ ]:
def obpi(asset, rf=0.1,floor = 0.9,vol = 0.5,t = 1):
  n = asset.shape[0]
  #   asset = list(asset)
  port_value = []
  risky_asset=[]
  risk_free_asset=[]
    
  port_value.append(asset[0])
  S = asset[0]
  K = asset[0]*floor
  nd1 = find_nd1(S=S,K=K,rf=rf,t=t,vol=vol)
  nd2 = find_nd2(S=S,K=K,rf=rf,t=t,vol=vol)
  num = S*nd1
  den = S*nd1 + K*np.exp(-rf*t)*nd2
  alpha = num/den
  risky_asset.append(asset[0]*alpha)
  risk_free_asset.append(port_value[0] - risky_asset[0])
  days = t*255  
  
  for i in range(1,n):
    port_value.append((asset[i]/asset[i-1])*risky_asset[i-1] + risk_free_asset[i-1]*(1+rf/255))
    S=asset[i]
    t = (days-i)/255
    nd1 = find_nd1(S=S,K=K,rf=rf,t=t,vol=vol)
    nd2 = find_nd2(S=S,K=K,rf=rf,t=t,vol=vol)
    num = S*nd1
    den = S*nd1 + K*np.exp(-rf*t)*nd2
    alpha = num/den
    risky_asset.append(asset[i]*alpha)
    risk_free_asset.append(port_value[i] - risky_asset[i])
      
  return port_value, risky_asset, risk_free_asset

In [ ]:
sensex_obpi = obpi(sensex_ad[:100])[0]

In [ ]:
df = sensex_ad[:100].to_frame(name='Sensex')
df['OBPI']=sensex_obpi
df.plot();